In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os # Para importar las imagenes

In [14]:
train_df = pd.read_csv('./csvs/train.csv')
train_df['path'] = train_df['id_code'].map(lambda x: os.path.join('./train_images/','{}.png'.format(x)))
train_df = train_df.drop(columns=['id_code'])
train_df

,diagnosis,path
0,2,./train_images/000c1434d8d7.png
1,4,./train_images/001639a390f0.png
2,1,./train_images/0024cdab0c1e.png
3,0,./train_images/002c21358ce6.png
4,0,./train_images/005b95c28852.png
...,...,...
3657,2,./train_images/ffa47f6a7bf4.png
3658,0,./train_images/ffc04fed30e6.png
3659,2,./train_images/ffcf7b45f213.png
3660,0,./train_images/ffd97f8cd5aa.png


In [15]:
test_df = pd.read_csv('./csvs/test.csv')
test_df['path'] = test_df['id_code'].map(lambda x: os.path.join('./test_images/','{}.png'.format(x)))
test_df = test_df.drop(columns=['id_code'])
test_df

,path
0,./test_images/0005cfc8afb6.png
1,./test_images/003f0afdcd15.png
2,./test_images/006efc72b638.png
3,./test_images/00836aaacf06.png
4,./test_images/009245722fa4.png
...,...
1923,./test_images/ff2fd94448de.png
1924,./test_images/ff4c945d9b17.png
1925,./test_images/ff64897ac0d8.png
1926,./test_images/ffa73465b705.png
